---
title: "1.9 Case Study: Modeling Network Flow using Systems of Equations"
subject: Linear Algebraic Systems
subtitle: Just go with the flow
short_title: "1.9 Case Study: Modeling Network Flow"
authors:
  - name: Renukanandan Tumu
    affiliations:
      - Dept. of Electrical and Systems Engineering
      - University of Pennsylvania
    email: nandant@seas.upenn.edu
license: CC-BY-4.0
keywords: sample notes, ese 2030, linear algebra
---

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/nikolaimatni/ese-2030/HEAD?labpath=/00_Linear_Algebraic_Systems/Case_Study_on_Network_Flows/027-application-network-flow.ipynb)

## What is network flow?

We can think about network flow as the flow of traffic through a road network, or water through a grid of pipes. 
In this section, we will try to answer questions about how much traffic or water moved through specific links in our network, and think about what would happen if we were to modify or delete those links.
A key assumption we will make is one of conservation. In the figure below, this means that $x_1+x_2 = 30$

Is this a reasonable assumption? In the case of cars, this means that the system starts with no cars, and that all of the cars that enter a system leave the system entirely, and do not stop at intersections. In the case of water pipes, it means that the pipes start dry, and that the pipes end dry. In the case of packets, it means we log all inflows and outflows, and that all of the packets

```{image} 02-net-flow.png
:alt: Network Flow
:width: 250px
:align: center
```

## What Linear Algebra tools do we need to model network flow?
### Concepts
1. Gaussian Elimination / Row Reduction


### Computation
1. Python
1. [numpy.array](https://numpy.org/doc/stable/reference/generated/numpy.array.html)
1. [sympy.matrices.matrices.MatrixReductions.echelon_form](https://docs.sympy.org/latest/modules/matrices/matrices.html#sympy.matrices.matrices.MatrixReductions.echelon_form)

## Small Example
Let's consider a system of pipes. We measure the inflows and outflows, and no water remains in the system after measurement.

```{image} 02-basic-network.png
:alt: Simple networks
:width: 300px
:align: center
```
> Adapted from LAA by Lay, Lay, and McDonald 


Because of our assumption that the total inflows and outflows are conserved, we can say that for each intersection $\text{inflow} - \text{outflow} = 0$. We can rearrange terms, and get the following equation, that  $\text{inflow} = \text{outflow}$. This is fairly intuitive, and we'll use this law to create our system of equations. We also know that the total outflows are equal to the total inflows, and we'll call this "total conservation".

```{math}
:label: simple-system
\begin{align}
\text{inflow} &= \text{outflow} \\
10 + 40 + 30 + 50 &= 60 + 30 + x_1  &[\text{total conservation}] \\
x_2 + x_3 &= x_1 + 30  &[\text{from } i_1] \\
10 + 40 &= x_2 + x_4   &[\text{from } i_2] \\
30 + 50 &= x_3 + x_5   &[\text{from } i_3] \\
x_5 + x_4 &= 60        &[\text{from } i_4] \\
\end{align}
```
Each equation above represents an equality that results from our original assumption. The first equation shows the total conservation, and the next 4 lines show the equations for each intersection.

We can represent this as an augmented matrix with some reorganization.

```{math}
:label: simple-system-reorg
\begin{align}
x_1 &= 40              &[\text{total conservation}] \\
-x_1 + x_2 + x_3 &= 30  &[\text{from } i_1] \\
x_2 + x_4 &= 10 + 40    &[\text{from } i_2] \\
x_3 + x_5 &= 30 + 50    &[\text{from } i_3] \\
x_5 + x_4 &= 60         &[\text{from } i_4] \\
\end{align}
```
By placing the coefficients for all of the terms in the equation, we can represent this as a matrix equation. Below, we have the expanded form of this equation.

```{math}
:label: simple-system-reorg
\begin{align}
\text{inflow} &= \text{outflow} \\
 1x_1 + 0x_2 + 0x_3 + 0x_4 + 0x_5 &= 40   &[\text{total conservation}] \\
-1x_1 + 1x_2 + 1x_3 + 0x_4 + 0x_5 &= 30   &[\text{from } i_1] \\
 0x_1 + 1x_2 + 0x_3 + 1x_4 + 0x_5 &= 50   &[\text{from } i_2] \\
 0x_1 + 0x_2 + 1x_3 + 0x_4 + 1x_5 &= 80   &[\text{from } i_3] \\
 0x_1 + 0x_2 + 0x_3 + 1x_4 + 1x_5 &= 60   &[\text{from } i_4] \\
\end{align}
```
In $Ax = b$ form, this is:
```{math}
\left[
\begin{array}{ccccc}
    %x_1 & x_2 &x_3 & x_4 &x_5\\\hline
    1 & 0 & 0 & 0 & 0 \\
    -1 & 1 & 1 & 0 & 0 \\
    0 & 1 & 0 & 1 & 0 \\
    0 & 0 & 1 & 0 & 1 \\
    0 & 0 & 0 & 1 & 1 \\
\end{array}
\right]
\left[
\begin{array}{c}
    x_1 \\ x_2 \\ x_3 \\ x_4 \\ x_5
\end{array}
\right]
=
\left[
\begin{array}{cccccc}
    40 \\
    30 \\
    50 \\
    80 \\
    60 \\
\end{array}
\right]
```



Finally, we have the augmented matrix below.
```{math}
\left[
\begin{array}{ccccc|c}
     &  & A &  & & b\\\hline
    1 & 0 & 0 & 0 & 0 &  40 \\
    -1 & 1 & 1 & 0 & 0 & 30 \\
    0 & 1 & 0 & 1 & 0 &  50 \\
    0 & 0 & 1 & 0 & 1 &  80 \\
    0 & 0 & 0 & 1 & 1 &  60 \\
\end{array}
\right]
```

In [11]:
import numpy as np
from numpy.linalg import solve
from sympy import Matrix, latex

In [12]:
augmented_matrix = np.array([
    [1 , 0 , 0 , 0,   0, 40],
    [-1 , 1 , 1 , 0 , 0, 30],
    [0 , 1 , 0 , 1, 0, 50],
    [0 , 0 , 1 , 0 , 1, 80],
    [0 , 0 , 0 , 1,   1, 60],
])
A = augmented_matrix[:,:-1]  # We take the first part of the augmented matrix, which represents A [: <- This means every row, :-1 <- This means all but the last column]
b = augmented_matrix[:, -1]  # Now the second part, which represents b [: <- This means every row, -1 <- This means only the last column]

In [13]:
matrix = Matrix(augmented_matrix)
matrix

Matrix([
[ 1, 0, 0, 0, 0, 40],
[-1, 1, 1, 0, 0, 30],
[ 0, 1, 0, 1, 0, 50],
[ 0, 0, 1, 0, 1, 80],
[ 0, 0, 0, 1, 1, 60]])

In [14]:
row_echelon_matrix = matrix.echelon_form(simplify=True)
row_echelon_matrix

Matrix([
[1, 0,  0,  0,  0,  40],
[0, 1,  1,  0,  0,  70],
[0, 0, -1,  1,  0, -20],
[0, 0,  0, -1, -1, -60],
[0, 0,  0,  0,  0,   0]])

Placing this in equation form again, we get the below. There is no pivot in the 5th column, so $x_5$ is free.

```{math}
\begin{align}
 1x_1 + 0x_2 + 0x_3 + 0x_4 + 0x_5 &= 40   \\
 0x_1 + 1x_2 + 1x_3 + 0x_4 + 0x_5 &= 70   \\
 0x_1 + 0x_2 + -1x_3 + 1x_4 + 0x_5 &= -20   \\
 0x_1 + 0x_2 + 0x_3 + -1x_4 + -1x_5 &= -60   \\
\end{align}
```
Simplifying, we get the following:

```{math}
\begin{align}
 x_1  &= 40        \\
 x_2 + x_3 &= 70   \\
 -x_3 + x_4 &= -20   \\
 x_4 + x_5 &= 60   \\
\end{align}
```
To get an expression to start, we can simplify the last equation to $$x_4 = 60 -x_5$$
Using back substitution, we can first substitute $x_4$ into our third equation, yielding
```{math}
x_3 - x_4 &= 20 \\
x_3 - (60 - x_5) &= 20 \\
x_3 &= 20 + 60 - x_5 \\
x_3 &= 80 - x_5
```
Again substituting $x_3$ into the second equation, we get:

```{math}
x_2 + x_3 &= 70 \\
x_2 + 80 - x_5 &= 70 \\
x_2 &= x_5 - 10
```
Gathering all of these equations, we get:
```{math}
\begin{align}
 x_1  &= 40        \\
 x_2  &= x_5 - 10   \\
 x_3  &= 80 - x_5  \\
 x_4 &= 60 - x_5   \\
 x_5 &= \text{free}
\end{align}
```
Now, we need to apply some human reasoning, namely that none of the traffic flow can be negative! This means that $x_5$ must be at least $10$, and at most $60$, which is due to $x_2$ and $x_4$ respectively. We set $x_5=30$, and get the following:
```{math}
\begin{align}
 x_1  &= 40   \\
 x_2  &= 20   \\
 x_3  &= 50   \\
 x_4  &= 30   \\
 x_5  &= 30
\end{align}
```


Let's check our solution in the original equation. We can do this via numpy as well, by using matrix multiplication. We will set our matrix $x$ based on our solution, and check that $Ax$ does equal $b$.
$$
x=\begin{bmatrix}
x_1 \\ x_2 \\ x_3 \\ x_4 \\ x_5
\end{bmatrix}
=\begin{bmatrix}
40 \\ 20 \\ 50 \\ 30 \\ 30
\end{bmatrix}
$$


In [15]:
x = np.array([40, 20, 50, 30, 30])
print(A@x)
print(b)
A@x == b

[40 30 50 80 60]
[40 30 50 80 60]


array([ True,  True,  True,  True,  True])

We got a solution!

## Penn Engineering Road Network
We can apply the same sort of thinking to roads!
```{image} 02-penn-roads.png
:alt: Penn Road Network
:width: 400px
:align: center
```


The above figure shows a road network that looks similar to that around Penn Engineering. We model these road networks the same way, by balancing the inflows and outflows of each node. We omit the total conservation equation, as none of our variables are in it. This yields the following equations:

```{math}
\begin{align}
\text{inflow} &= \text{outflow} \\
x_6 + x_2 &= x_1       &[\text{from } i_1]\\
x_3 + x_5 &= x_2 + 50  &[\text{from } i_2]\\ 
x_4 + 60 &= x_3        &[\text{from } i_3]\\

x_8 &= x_4 + x_9          &[\text{from } i_4]\\
80 + x_7 &= x_{5} +x_8    &[\text{from } i_5]\\
100 &= x_{6} + x_7        &[\text{from } i_6]\\

100 + 80 + 60 &= x_1 + 50 + x_9 &[\text{total conservation}]\\

\end{align}
```
To put this in matrix form, we need to do the same as before, and reorganize terms.

```{math}
\begin{align}
\text{inflow} &= \text{outflow} \\
-x_1 + x_2 + x_6 &= 0       &[\text{from } i_1]\\
- x_2 + x_3 + x_5 &= 50     &[\text{from } i_2]\\ 
-x_3 + x_4 &= -60           &[\text{from } i_3]\\

x_4 - x_8 + x_9 &= 0         &[\text{from } i_4]\\
x_{5} -x_7 + x_8 &= 80       &[\text{from } i_5]\\
x_{6} + x_7 &= 100           &[\text{from } i_6]\\

x_1 + x_9 &= 190 &[\text{total conservation}]\\
\end{align}
```

This gives us the following matrix
```{math}
\left[
\begin{array}{ccccccccc|c}
    x_1 & x_2 & x_3 & x_4 & x_5 & x_6 & x_7 & x_8 & x_9 & C \\
    \hline
    -1  & 1   & 0   & 0   & 0   & 1   & 0   & 0   & 0   & 0 \\
    0   & -1  & 1   & 0   & 1   & 0   & 0   & 0   & 0   & 50 \\
    0   & 0   & -1  & 1   & 0   & 0   & 0   & 0   & 0   & -60 \\
    
    0   & 0   & 0   & 1   & 0   & 0   & 0   & -1  & 1   & 0 \\
    0   & 0   & 0   & 0   & 1   & 0   & -1  & 1   & 0   & 80 \\
    0   & 0   & 0   & 0   & 0   & 1   & 1   & 0   & 0   & 100 \\
    
    1   & 0   & 0   & 0   & 0   & 0   & 0   & 0   & 1   & 190 \\
\end{array}
\right]
\begin{array}{l}
\\
[\text{from } i_1]\\
[\text{from } i_2]\\
[\text{from } i_3]\\
[\text{from } i_4]\\
[\text{from } i_5]\\
[\text{from } i_6]\\
[\text{total conservation}]\\
\end{array}
```

In [40]:
augmented_matrix = np.array([
        [-1 , 1 , 0 , 0 , 0 , 1 , 0 , 0 , 0 , 0],
        [0 , -1 , 1 , 0 , 1 , 0 , 0 , 0 , 0 , 50],
        [0 , 0 , -1 , 1 , 0 , 0 , 0 , 0 , 0 , -60],
        [0 , 0 , 0 , 1 , 0 , 0 , 0 , -1 , 1 , 0],
        [0 , 0 , 0 , 0 , 1 , 0 , -1 , 1 , 0 , 80],
        [0 , 0 , 0 , 0 , 0 , 1 , 1 , 0 , 0 , 100],
        [1 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 1 , 190],
])
A = augmented_matrix[:,:-1]  # We take the first part of the augmented matrix, which represents A [: <- This means every row, :-1 <- This means all but the last column]
b = augmented_matrix[:, -1]  # Now the second part, which represents b [: <- This means every row, -1 <- This means only the last column]

In [41]:
matrix = Matrix(augmented_matrix)
echelon_form = matrix.echelon_form()
echelon_form

Matrix([
[-1,  1,  0, 0, 0, 1,  0,  0, 0,   0],
[ 0, -1,  1, 0, 1, 0,  0,  0, 0,  50],
[ 0,  0, -1, 1, 0, 0,  0,  0, 0, -60],
[ 0,  0,  0, 1, 0, 0,  0, -1, 1,   0],
[ 0,  0,  0, 0, 1, 0, -1,  1, 0,  80],
[ 0,  0,  0, 0, 0, 1,  1,  0, 0, 100],
[ 0,  0,  0, 0, 0, 0,  0,  0, 0,   0]])

Let's print the row-echelon matrix in a nicer way:
```{math}
\left[
\begin{array}{ccccccccc|c}
    x_1 & x_2 & x_3 & x_4 & x_5 & x_6 & x_7 & x_8 & x_9 & C \\
    \hline
-1 & 1 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0\\
0 & -1 & 1 & 0 & 1 & 0 & 0 & 0 & 0 & 50\\
0 & 0 & -1 & 1 & 0 & 0 & 0 & 0 & 0 & -60\\
0 & 0 & 0 & 1 & 0 & 0 & 0 & -1 & 1 & 0\\
0 & 0 & 0 & 0 & 1 & 0 & -1 & 1 & 0 & 80\\
0 & 0 & 0 & 0 & 0 & 1 & 1 & 0 & 0 & 100\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0
\end{array}
\right]
```

Looking at the row-echelon form, we should notice a few things. First, that the last row is empty. Second, that there are not pivots in every column. In the columns for $x_7, x_{8}, x_{9}$, there are no pivots. This means there are $3$ free variables in this system. This makes sense, because we have only $6$ intersections, giving $6$ equations, but $9$ flows, representing $9$ variables. In order to find a solution, let's set all of those free variables to be zero. We can think of this as closing all of the roads that are free variables.

Let's solve for the free variables in terms of the basic variables:

First for $x_7$:
```{math}
x_6 + x_7 &= 100 \\
x_7 &= 100 - x_6
```
We know that if $x_7>100$, then $x_6$ will be negative.

Now for $x_{8}$:
```{math}
x_5 - x_7 + x_8 &= 80 \\
x_8 &= 80 - x_5 + x_7
```
We also know that $x_8$ cannot be more than $80$.
 
For $x_{9}$:
```{math}
x_4 - x_8 + x_9 &= 0 \\
x_9 &= -x_4 + x_8
```     

Now let's solve for the system, setting $x_7, x_8, x_9$ to be equal to $0$.
To do this, we first select all of the columns from the augmented matrix which correspond to basic variables. This is because if a variable is equal to zero, then we can subtract them from an equation without impact. We can think of this as closing the roads that correspond to $x_7$, $x_8$, and $x_9$.

In [42]:
smaller_augmented_matrix = np.array(echelon_form[:,[0,1,2,3,4,5,9]]).astype(np.float64)
"""
Above, we just selected all of the rows that correspond to the non-free variables. 
If the free variables are zero, then we can freely subtract them from the rest of the equations.
"""
# smaller_augmented_matrix = np.vstack([smaller_augmented_matrix, 
#                                       np.array([[0,0,0,0,0,0,0,0,1,0,80]]),
#                                      ])
smaller_augmented_matrix = smaller_augmented_matrix[:-1,:]
smaller_augmented_matrix

array([[ -1.,   1.,   0.,   0.,   0.,   1.,   0.],
       [  0.,  -1.,   1.,   0.,   1.,   0.,  50.],
       [  0.,   0.,  -1.,   1.,   0.,   0., -60.],
       [  0.,   0.,   0.,   1.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   1.,   0.,  80.],
       [  0.,   0.,   0.,   0.,   0.,   1., 100.]])

Now that there are pivots in all columns, we can try to use numpy's `solve` method to get a solution.

In [43]:
sA = smaller_augmented_matrix[:,:-1]  # We take the first part of the augmented matrix, which represents A [: <- This means every row, :-1 <- This means all but the last column]
sb = smaller_augmented_matrix[:, -1]  # Now the second part, which represents b [: <- This means every row, -1 <- This means only the last column]

In [44]:
x = solve(sA,sb)
x

array([190.,  90.,  60.,   0.,  80., 100.])

Let's write these solutions down first.
```{math}
\begin{align}
x_1 &= 190 \\
x_2 &= 90 \\
x_3 &= 60 \\
x_4 &= 0 \\
x_5 &= 80 \\
x_6 &= 100 \\
\end{align}
```

One key thing to remember is that these variables are currently unconstrained. In our road network, a negative number means that traffic is flowing the wrong way. Let's verify that the free variables we specified are actually zero, and see what we can learn from each solution.

```{math}
x_7 &= 100 - x_6 \\
x_7 &= 0
```

```{math}
x_8 &= 80 - x_5 + x_7 \\
x_8 &= 80 - 80 + 0 \\
&= 0 
```


```{math}
x_9 &= -x_4 + x_8 \\
x_9 &= 0 + 0 \\
&= 0 
```


Now let's check our solution:

$$
x=\begin{bmatrix}
x_1 \\ x_2 \\ x_3 \\ x_4 \\ x_5 \\ x_6 \\ x_7 \\ x_8 \\ x_9
\end{bmatrix}
=\begin{bmatrix}
190 \\ 90 \\ 60 \\ 0 \\ 80 \\ 100 \\ 0 \\ 0 \\ 0
\end{bmatrix}
$$


In [47]:
x = np.array([190,90,60,0,80,100,0,0,0])
print(A@x)
print(b)
print(A@x == b)

[  0  50 -60   0  80 100 190]
[  0  50 -60   0  80 100 190]
[ True  True  True  True  True  True  True]


As an extension, if you want to try to solve the same system, but fix the values for the free variables to different values. One way to do this would be to add equations to the system of equations that fix the values of specific variables. For example, you could add a row fixing $x_7 = 10$ like so:
$$
\left[
\begin{array}{ccccccccc|c}
    x_1 & x_2 & x_3 & x_4 & x_5 & x_6 & x_7 & x_8 & x_9 & C \\
    \hline
    0   & 0   & 0   & 0   & 0   & 0   & 1 & 0 & 0 & 10\\
\end{array}
\right]
$$


## Related Resources
If you're interested in modeling traffic flow, check out the [Cellular Transmission Model](https://connected-corridors.berkeley.edu/planning-system/assessing-potential-project-benefits-analysis-modeling-and-simulation/phases-ams-2) and [Networked Macroscopic Fundamental Diagram](https://arxiv.org/abs/2406.10433) models for more info on how traffic flow is modeled in practice, and some approaches to control traffic lights.

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/nikolaimatni/ese-2030/HEAD?labpath=/00_Linear_Algebraic_Systems/Case_Study_on_Network_Flows/027-application-network-flow.ipynb)